In [ ]:
import numpy as np
import pandas as pd
import os
import string

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from IPython.display import HTML

def create_download_link(title = "Download a file", filename = "file"):
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

/kaggle/input/hse-text-authorship-challenge/submission.csv
/kaggle/input/books-eng/web2.txt
/kaggle/input/books-eng/result.pq
/kaggle/input/books-eng/books2.pq
/kaggle/input/books-eng/books_f.pq


Объединю спарсенные книги в один датафрейм

Прочитаю первый фрейм

In [ ]:
books_f = pd.read_parquet('/kaggle/input/books-eng/books_f.pq')

Обрежу текстовые вставки в текстах с сайта royallib

In [ ]:
def royallib_remove(row):
    try:
        row = row.split('÷òåíèÿ!')[1]
        row = row.split('Ñïàñèáî, ÷òî ñêà÷àëè êíèãó â áåñïëàòíîé ýëåêòðîííîé áèáëèîòåêå')[0]
        return row
    except:
        return row

In [ ]:
books_f.text = books_f.text.apply(lambda x: royallib_remove(x))

/tmp/ipykernel_30/2546441832.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  books_f.text = books_f.text.apply(lambda x: royallib_remove(x))


Получу из имён файлов признаки author и title

In [ ]:
books_f['title'] = books_f['FileName'].apply(lambda x: x.split('-')[1])
books_f['title'] = books_f['title'].apply(lambda x: x[:-4])
books_f['title'] = books_f['title'].apply(lambda x: x.replace('_', ' '))
books_f['title'] = books_f['title'].apply(lambda x: string.capwords(x))
books_f['title'] = books_f['title'].apply(lambda x: x.replace('-', ' '))
books_f['title'] = books_f['title'].apply(lambda x: x.replace(',', ''))
books_f['title'] = books_f['title'].apply(lambda x: x.replace('.', ' '))
books_f['title'] = books_f['title'].apply(lambda x: x.replace('  ', ' '))
books_f['title'] = books_f['title'].apply(lambda x: x.replace('  ', ' '))
books_f['title'] = books_f['title'].apply(lambda x: x.replace(' ', '_'))

books_f['author'] = books_f['FileName'].apply(lambda x: x.split('-')[0])
books_f['author'] = books_f['author'].apply(lambda x: x.replace('_', ' '))
books_f['author'] = books_f['author'].apply(lambda x: string.capwords(x))
books_f['author'] = books_f['author'].apply(lambda x: x.replace('-', ' '))
books_f['author'] = books_f['author'].apply(lambda x: x.replace(',', ''))
books_f['author'] = books_f['author'].apply(lambda x: x.replace('.', ' '))
books_f['author'] = books_f['author'].apply(lambda x: x.replace('  ', ' '))
books_f['author'] = books_f['author'].apply(lambda x: x.replace('  ', ' '))
books_f['author'] = books_f['author'].apply(lambda x: x.replace(' ', '_'))

Получу признак author_sn с фамилий автора, для более удобной сверки, так как часто ФИО авторов бывают записаны по разному

In [ ]:
books_f['author_sn'] = books_f.author.apply(lambda x: x.split('_')[-1])

In [ ]:
books_f.columns

Index(['FileName', 'author', 'title', 'text', 'sn', 'is_gutenberg',
       'author_sn'],
      dtype='object')

In [ ]:
books_f = books_f[['author', 'author_sn', 'title', 'text', 'FileName', 'is_gutenberg']]

In [ ]:
books_f

,author,author_sn,title,text,FileName,is_gutenberg
0,Mark_Meadows,Meadows,Joy_Ride,Transcriber's Note:\n...,Mark_Meadows-Joy_Ride.txt,0
1,Goldfrap,Goldfrap,Boy_Scouts_For_Uncle_Sam,THE BOY SCOUTS FOR UNCLE SAM\n\nby\n\nLIEUT. H...,Goldfrap-Boy_Scouts_for_Uncle_Sam.txt,0
2,Henry_Cabot_Lodge,Lodge,Hero_Tales_From_American_History,HERO TALES FROM AMERICAN HISTORY\n\nBy Henry C...,Henry_Cabot_Lodge-Hero_Tales_from_American_His...,0
3,G_K_Chesterton,Chesterton,The_Defendant,THE DEFENDANT\n\nBY G. K. CHESTERTON\n\nAUTHOR...,G_K_Chesterton-The_Defendant.txt,0
4,Dorothy_Scarborough,Scarborough,Famous_Modern_Ghost_Stories,"FAMOUS MODERN GHOST STORIES\n\nSelected, with ...",Dorothy_Scarborough-Famous_Modern_Ghost_Storie...,0
...,...,...,...,...,...,...
3352,William_Faulkner,Faulkner,The_Sound_And_The_Fury,William Faulkner\n\n\nThe Sound and the Fury\n...,William_Faulkner-The_Sound_and_the_Fury.txt,0
3353,William_Faulkner,Faulkner,Absalom_Absalom,"Absalom, Absalom!\n\n\nWilliam Faulkner\n\n\n\...",William_Faulkner-Absalom_Absalom.txt,0
3354,William_Faulkner,Faulkner,Mosquitoes,William Faulkner\n\n\nMosquitoes\n\nTo Helen\n...,William_Faulkner-Mosquitoes.txt,0
3355,Tolkien,Tolkien,Two_Towers,John R. R. Tolkien\n\nTwo Towers\n\n\n\nBook I...,Tolkien-Two_Towers.txt,0


In [ ]:
books_f.text.iloc[3356]

"\n\n\n\n\n\nFROM THE TREE TO THE LABYRINTH\n\n\n\n\nFROM THE TREE TO THE LABYRINTH\n\n\n\nHISTORICAL STUDIES ON THE SIGN AND INTERPRETATION\n\nUMBERTO ECO\n\nTranslated by Anthony Oldcorn\n\n\n\n\nHarvard University Press\n\nCambridge, Massachusetts\n\nLondon, England\n\n2014\n\n\n\nCopyright © 2014 by the President and Fellows of Harvard College\n\nAll rights reserved\n\nOriginally published as Dall’albero al labirinto: Studi storici sul segno e l’interpretazione, by Umberto Eco, copyright © 2007 RCS Libri S.p.A.\n\nChapters 11 and 12, “The Language of the Austral Land” and “The Linguistics of Joseph de Maistre,” were originally published in Serendipities, by Umberto Eco, translated by William Weaver. Copyright © 1998 by Columbia University Press. Reprinted with permission of the publisher.\n\nJacket design: Graciela Galup\n\nJacket art: Magdolna Ban, Mystic (2004), oil on canvas. Private collection / The Bridgeman Art Library.\n\nThe Library of Congress has cataloged the printed edi

In [ ]:
books_f.to_parquet('books_f.pq')
create_download_link(filename='books_f.pq')

Прочитаю второй фрейм

In [ ]:
books2 = pd.read_parquet('/kaggle/input/books-eng/books2.pq')

Добавлю признак FileName

In [ ]:
books2['FileName'] = books2.author + "-" + books2.title + ".txt"

Уберу из имен авторов 'et al.' в конце

In [ ]:
books2['author_origin'] = books2.author_origin.apply(lambda x: x[:-7] if 'et al.' in x else x)

Создам признаки title и author

In [ ]:
books2['title'] = books2['title_origin'].apply(lambda x: x.replace(',', ''))
books2['title'] = books2['title'].apply(lambda x: x.replace('.', ' '))
books2['title'] = books2['title'].apply(lambda x: x.replace('-', ' '))
books2['title'] = books2['title'].apply(lambda x: x.replace('  ', ' '))
books2['title'] = books2['title'].apply(lambda x: x.replace('  ', ' '))
books2['title'] = books2['title'].apply(lambda x: string.capwords(x))
books2['title'] = books2['title'].apply(lambda x: x.replace(' ', '_'))

books2['author'] = books2['author_origin'].apply(lambda x: x.replace(',', ''))
books2['author'] = books2['author'].apply(lambda x: x.replace('(', ''))
books2['author'] = books2['author'].apply(lambda x: x.replace(')', ''))
books2['author'] = books2['author'].apply(lambda x: x.replace('.', ' '))
books2['author'] = books2['author'].apply(lambda x: x.replace('-', ' '))
books2['author'] = books2['author'].apply(lambda x: x.replace('  ', ' '))
books2['author'] = books2['author'].apply(lambda x: x.replace('  ', ' '))
books2['author'] = books2['author'].apply(lambda x: string.capwords(x))
books2['author'] = books2['author'].apply(lambda x: x.replace(' ', '_'))

Создам признак author_sn

In [ ]:
books2['author_sn'] = books2.author.apply(lambda x: x.split('_')[-1])

Уберу лишние фрагменты в текстах

In [ ]:
def xxx_remove(row):
    try:
        row = row.split(' ***')[1]
        row = row.split('End of Project Gutenberg')[0]
        return row
    except:
        return row

In [ ]:
books2.text = books2.text.apply(lambda x: xxx_remove(x))

In [ ]:
books2.to_parquet('books2.pq')
create_download_link(filename='books2.pq')

Приведу фреймы к одному виду для склейки

In [ ]:
books = books2[['author', 'author_sn', 'title', 'text', 'FileName', 'is_gutenberg']]

Объединю оба фрейма в один по высоте

In [ ]:
result = pd.concat([books_f, books], ignore_index=True)

In [ ]:
result.shape

(9393, 6)

Уберу дубликаты

In [ ]:
result = result.dropna()

In [ ]:
result.shape

(9357, 6)

In [ ]:
Уберу дубликаты по признакам автор и название

In [ ]:
result = result.drop_duplicates(subset=['author', 'title'])

In [ ]:
result.shape

(8967, 6)

In [ ]:
result = result.sort_values('author')

In [ ]:
result

,author,author_sn,title,text,FileName,is_gutenberg
4878,A_A_Milne,Milne,Not_That_It_Matters,"\r\n\r\n\r\nScanned by Charles Aldarondo, text...",A_A_Milne-Not_That_It_Matters.txt,1
5832,A_A_Milne,Milne,Once_A_Week,"\r\n\r\n\r\n\r\nProduced by Malcolm Farmer, St...",A_A_Milne-Once_A_Week.txt,1
923,A_A_Milne,Milne,The_Red_House_Mystery,THE RED HOUSE MYSTERY\n\nBy A. A. Milne\n\n\n\...,a_A_Milne-The_Red_House_Mystery.txt,0
2470,A_Alpheus,Alpheus,Complete_Hypnotism,MIND-READING AND SPIRITUALISM***\n\n\nE-text p...,a_Alpheus-Complete_Hypnotism.txt,0
5308,A_B_Muzzey,Muzzey,The_Young_Maiden,\r\n\r\nE-text prepared by Roger Frank and the...,A_B_Muzzey-The_Young_Maiden.txt,1
...,...,...,...,...,...,...
5157,`abdu'l_Bahá,Bahá,Bahá'í_World_Faith,\r\n\r\n\r\n\r\n\r\n\r\nBahá'í World Faith\r\n...,`abdu'l_Bahá-Bahá'í_World_Faith.txt,1
5158,`abdu'l_Bahá,Bahá,A_Traveler's_Narrative_Written_To_Illustrate_T...,\r\n\r\n\r\n\r\n\r\n\r\nA Traveler's Narrative...,`abdu'l_Bahá-A_Traveler's_Narrative_Written_To...,1
4543,À_Kempis_Thomas,Thomas,The_Imitation_Of_Christ,\r\n\r\n\r\n\r\n\r\nThe Imitation of Christ\r\...,À_Kempis_Thomas-The_Imitation_Of_Christ.txt,1
6595,Émile_Faguet,Faguet,Initiation_Into_Philosophy,"</p>\n\n<p id=""id00013"" style=""margin-top: 4em...",Émile_Faguet-Initiation_Into_Philosophy.txt,1


Сохраню, полученный датафрейм

In [ ]:
result.to_parquet('result.pq')
create_download_link(filename='result.pq')

Из полученного датафрейма result.pq выберу только тексты книг 101 автора из списка полученного по мотивам топ-100 (https://www.imdb.com/list/ls005774742/). Добавил данный список авторов (AAA.xlsx) в датасет (books-eng).

In [ ]:
authors = pd.read_excel('/kaggle/input/books-eng/AAA.xlsx', header = None)

In [ ]:
authors = authors.rename({0 : 'authors'},axis=1)

In [ ]:
authors['authors'] = authors['authors'].apply(lambda x: x.replace('-', ' '))

In [ ]:
authors['authors'] = authors['authors'].apply(lambda x: string.capwords(x))

In [ ]:
authors = authors.apply(lambda x: x.str.replace(' ', '_'))

In [ ]:
authors['sn'] = authors.authors.apply(lambda x: x if x.split('_')[-1] == '' else x.split('_')[-1])

Получил датфрейм со списком авторов из рейтинга top100

In [ ]:
authors

,authors,sn
0,Aesop,Aesop
1,Ahmet_Hamdi_Tanpinar,Tanpinar
2,Albert_Camus,Camus
3,Aldous_Huxley,Huxley
4,Alexander_Pushkin,Pushkin
...,...,...
96,William_Faulkner,Faulkner
97,William_Shakespeare,Shakespeare
98,Yasar_Kemal,Kemal
99,Agatha_Christie,Christie


In [ ]:
result = pd.read_parquet('/kaggle/input/books-eng/result.pq')

Посмотрю каких авторов не хватает

In [ ]:
auths = authors[~authors.authors.isin(res.author)]

In [ ]:
auths

,authors,sn
35,Hans_Christian_Andersen,Andersen
46,J_R_R_Tolkien,Tolkien


Проверю наличие книг Толкиена по фамилии

In [ ]:
result['Tolkien' == result.author_sn]

,author,author_sn,title,text,FileName,is_gutenberg
3344,Tolkien,Tolkien,The_Hobbit,\n\n\n\n\n\nJ.R.R. Tolkien. The Hobbit\n\n\n\n...,Tolkien-The_Hobbit.txt,0
3343,Tolkien,Tolkien,The_Return_Of_The_King,John R. R. Tolkien\n\nThe Return of the King\n...,Tolkien-The_Return_of_the_King.txt,0
3342,Tolkien,Tolkien,Fellowship_Of_The_Ring,John R. R. Tolkien\n\nFellowship of the Ring\n...,Tolkien-Fellowship_of_the_Ring.txt,0
3341,Tolkien,Tolkien,The_Silmarillon,\n\n\n\n\n\nJ.R.R. Tolkien. The Silmarillon\n\...,Tolkien-The_Silmarillon.txt,0
3355,Tolkien,Tolkien,Two_Towers,John R. R. Tolkien\n\nTwo Towers\n\n\n\nBook I...,Tolkien-Two_Towers.txt,0


In [ ]:
a_list = list(auths.sn)
a_list

['Andersen', 'Tolkien']

Проверю наличие недостающих авторов по их фамилиям

In [ ]:
res2 = result[result.author_sn.isin(auths.sn)]

In [ ]:
res2

,author,author_sn,title,text,FileName,is_gutenberg
4127,Hans_Christian_Andersen,Andersen,Stories_From_Hans_Andersen,"\r\n\r\n\r\n\r\nProduced by Stacy Brown, Jason...",H_C_Andersen-Stories_From_Hans_Andersen.txt,1
3651,Hans_Christian_Andersen,Andersen,A_Christmas_Greeting:_A_Series_Of_Stories,\r\n\r\n\r\n\r\n\r\nNew Juveniles for 1864\r\n...,H_C_Andersen-A_Christmas_Greeting:_A_Series_Of...,1
3471,Hans_Christian_Andersen,Andersen,Andersen's_Fairy_Tales,\r\n\r\n\r\n\r\nProduced by Dianne Bean\r\n\r\...,H_C_Andersen-Andersen's_Fairy_Tales.txt,1
3344,J_R_R_Tolkien,Tolkien,The_Hobbit,\n\n\n\n\n\nJ.R.R. Tolkien. The Hobbit\n\n\n\n...,Tolkien-The_Hobbit.txt,0
3343,J_R_R_Tolkien,Tolkien,The_Return_Of_The_King,John R. R. Tolkien\n\nThe Return of the King\n...,Tolkien-The_Return_of_the_King.txt,0
3342,J_R_R_Tolkien,Tolkien,Fellowship_Of_The_Ring,John R. R. Tolkien\n\nFellowship of the Ring\n...,Tolkien-Fellowship_of_the_Ring.txt,0
3341,J_R_R_Tolkien,Tolkien,The_Silmarillon,\n\n\n\n\n\nJ.R.R. Tolkien. The Silmarillon\n\...,Tolkien-The_Silmarillon.txt,0
3355,J_R_R_Tolkien,Tolkien,Two_Towers,John R. R. Tolkien\n\nTwo Towers\n\n\n\nBook I...,Tolkien-Two_Towers.txt,0


Поправлю написание ФИО авторов для соответствия между датафремами

In [ ]:
condition = result['author'] == 'H_C_Andersen'
result.loc[condition, 'author'] = 'Hans_Christian_Andersen'

In [ ]:
condition = result['author'] == 'Tolkien'
result.loc[condition, 'author'] = 'J_R_R_Tolkien'

In [ ]:
result.shape

(8965, 6)

In [ ]:
result = result[~(result.author == 'Andersen')]

In [ ]:
result = result[~(result.author == 'Hc_Andersen')]

In [ ]:
res2.author.value_counts()

author
Mark_Twain               43
Charles_Dickens          30
Alexandre_Dumas          28
Arthur_Conan_Doyle       25
Jules_Verne              22
                         ..
Francois_Rabelais         1
Federico_Garcia_Lorca     1
Ernest_Hemingway          1
Erich_Maria_Remarque      1
Yasar_Kemal               1
Name: count, Length: 173, dtype: int64

Проверю есть ли книги всех авторов в финальном датафрейме

In [ ]:
res = result[result.author.isin(authors.authors)]

In [ ]:
res

,author,author_sn,title,text,FileName,is_gutenberg
1865,Aesop,Aesop,The_Aesop_For_Children,THE ÆSOP FOR CHILDREN\n\n[Illustration: THE CO...,Aesop-The_Aesop_for_Children.txt,0
5582,Aesop,Aesop,Aesop's_Fables,\r\n\r\n[Illustration]\r\n\r\n\r\n\r\n\r\nThe ...,Aesop-Aesop's_Fables.txt,1
3937,Agatha_Christie,Christie,The_Mysterious_Affair_At_Styles,\r\n\r\n\r\n\r\n\r\nThe Mysterious Affair at S...,Agatha_Christie-The_Mysterious_Affair_At_Style...,1
3943,Agatha_Christie,Christie,The_Man_In_The_Brown_Suit,\r\n\r\n\r\n\r\n THE MAN IN THE BROWN SUIT\r\...,Agatha_Christie-The_Man_In_The_Brown_Suit.txt,1
3939,Agatha_Christie,Christie,The_Murder_On_The_Links,\r\n\r\n\r\n\r\n\r\ncover\r\n\r\n\r\n\r\n\r\nT...,Agatha_Christie-The_Murder_On_The_Links.txt,1
...,...,...,...,...,...,...
1750,William_Shakespeare,Shakespeare,King_Lear,THE TRAGEDY OF KING LEAR\n\nby William Shakesp...,William_Shakespeare-King_Lear.txt,0
3295,William_Shakespeare,Shakespeare,Cymbeline,1609\n\nCYMBELINE\n\nby William Shakespeare\n\...,William_Shakespeare-Cymbeline.txt,0
2847,William_Shakespeare,Shakespeare,Julius_Caesar,1599\n\n\nTHE TRAGEDY OF JULIUS CAESAR\n\nby W...,William_Shakespeare-Julius_Caesar.txt,0
3319,William_Shakespeare,Shakespeare,Complete_Works_Second_Edition,THE RSC SHAKESPEARE COMPLETE WORKS\nTHE TEMPES...,William_Shakespeare-Complete_Works_Second_Edit...,0


In [ ]:
res.author.value_counts()

author
Mark_Twain               43
Charles_Dickens          30
Alexandre_Dumas          28
Arthur_Conan_Doyle       25
Jules_Verne              22
                         ..
Federico_Garcia_Lorca     1
Ernest_Hemingway          1
Mikhail_Lermontov         1
J_D_Salinger              1
Yasar_Kemal               1
Name: count, Length: 101, dtype: int64

Сохраню датафрейм с книгами топ100 авторов

In [ ]:
res.to_parquet('top101.pq')
create_download_link(filename='top101.pq')

Добавил полученный датафрейм с топ101 авторами в датасет.

In [ ]:
top101 = pd.read_parquet('/kaggle/input/books-eng/top100.pq')

In [ ]:
top101

,author,author_sn,title,text,FileName,is_gutenberg
1865,Aesop,Aesop,The_Aesop_For_Children,THE ÆSOP FOR CHILDREN\n\n[Illustration: THE CO...,Aesop-The_Aesop_for_Children.txt,0
5582,Aesop,Aesop,Aesop's_Fables,\r\n\r\n[Illustration]\r\n\r\n\r\n\r\n\r\nThe ...,Aesop-Aesop's_Fables.txt,1
3937,Agatha_Christie,Christie,The_Mysterious_Affair_At_Styles,\r\n\r\n\r\n\r\n\r\nThe Mysterious Affair at S...,Agatha_Christie-The_Mysterious_Affair_At_Style...,1
3943,Agatha_Christie,Christie,The_Man_In_The_Brown_Suit,\r\n\r\n\r\n\r\n THE MAN IN THE BROWN SUIT\r\...,Agatha_Christie-The_Man_In_The_Brown_Suit.txt,1
3939,Agatha_Christie,Christie,The_Murder_On_The_Links,\r\n\r\n\r\n\r\n\r\ncover\r\n\r\n\r\n\r\n\r\nT...,Agatha_Christie-The_Murder_On_The_Links.txt,1
...,...,...,...,...,...,...
1750,William_Shakespeare,Shakespeare,King_Lear,THE TRAGEDY OF KING LEAR\n\nby William Shakesp...,William_Shakespeare-King_Lear.txt,0
3295,William_Shakespeare,Shakespeare,Cymbeline,1609\n\nCYMBELINE\n\nby William Shakespeare\n\...,William_Shakespeare-Cymbeline.txt,0
2847,William_Shakespeare,Shakespeare,Julius_Caesar,1599\n\n\nTHE TRAGEDY OF JULIUS CAESAR\n\nby W...,William_Shakespeare-Julius_Caesar.txt,0
3319,William_Shakespeare,Shakespeare,Complete_Works_Second_Edition,THE RSC SHAKESPEARE COMPLETE WORKS\nTHE TEMPES...,William_Shakespeare-Complete_Works_Second_Edit...,0
